In [ ]:
import pandas as pd
import os
import sys
import numpy as np

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
# Set the current working directory to the project root
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
os.chdir(ROOT_DIR)

In [ ]:
import pandas as pd
import json
from tqdm.auto import tqdm
import time
import os

from src.llm.chains import create_narrative_generator_chain, create_subnarrative_generator_chain

In [ ]:
from src.data_management.label_parser import parse_json_for_narratives_subnarratives
import pandas as pd

print("--- Updating label formats in source CSVs ---")

# --- Load data for augmentation ---
print("\n--- Loading data for augmentation ---")

print("Loading target narratives...")
target_narratives_df = pd.read_csv("least_perf_narratives.csv")

print("Loading target subnarratives...")
target_subnarratives_df = pd.read_csv("least_perf_subnarratives.csv")

target_narratives_list = target_narratives_df['label'].tolist()
target_subnarratives_list = target_subnarratives_df['label'].tolist()

narrative_definitions_df = pd.read_csv("data/narrative_definitions.csv")
subnarrative_definitions_df = pd.read_csv("data/subnarrative_definitions.csv")

print("Data loaded.")



In [ ]:
target_narratives_df['definition'] = target_narratives_df['label'].apply(
    lambda x: narrative_definitions_df[narrative_definitions_df['narrative'] == x]['definition'].values[0] if x in narrative_definitions_df['narrative'].values else None
)

target_narratives_df['examples'] = target_narratives_df['label'].apply(
    lambda x: narrative_definitions_df[narrative_definitions_df['narrative'] == x]['example'].values[0] if x in narrative_definitions_df['narrative'].values else None
)

target_subnarratives_df['definition'] = target_subnarratives_df['label'].apply(
    lambda x: subnarrative_definitions_df[subnarrative_definitions_df['subnarrative'] == x]['definition'].values[0] if x in subnarrative_definitions_df['subnarrative'].values else None
)

target_subnarratives_df['examples'] = target_subnarratives_df['label'].apply(
    lambda x: subnarrative_definitions_df[subnarrative_definitions_df['subnarrative'] == x]['examples'].values[0] if x in subnarrative_definitions_df['subnarrative'].values else None
)

In [ ]:
NUM_EXAMPLES_PER_LABEL = 5

narrative_generator = create_narrative_generator_chain()
subnarrative_generator = create_subnarrative_generator_chain()

In [ ]:
# from tqdm.auto import tqdm

# generated_narrative_texts = []

# for idx, row in tqdm(target_narratives_df.iterrows(), total=len(target_narratives_df)):
#     print(f"Generating narratives for label: {row['label']}")
#     texts = []
#     # Prepare input for the generator
#     input_dict = {
#         "narrative_name": row['label'],
#         "narrative_def": row['definition'],
#         "num_examples": NUM_EXAMPLES_PER_LABEL,
#         "narrative_example": row['examples'] if pd.notnull(row['examples']) else "",
#         "format_instructions": "Return only the generated text as a string."
#     }
    
#     result = narrative_generator.invoke(input_dict)
#     # Extract the generated text from the result
#     articles = result.articles  # Access the articles from the Pydantic object
#     texts = [article.generated_text for article in articles]  # Extract the generated text from each article
#     generated_narrative_texts.append(texts)

# # Add the generated texts to the DataFrame
# target_narratives_df['generated_texts'] = generated_narrative_texts



In [ ]:
# # Prepare data for JSON export
# narrative_texts_json = []
# for i, row in target_narratives_df.iterrows():
#     narrative_texts_json.append({
#         "narrative": row["label"],
#         "generated_texts": row["generated_texts"]
#     })

# # Save to file
# with open("generated_narrative_texts.json", "w", encoding="utf-8") as f:
#     json.dump(narrative_texts_json, f, ensure_ascii=False, indent=2)

In [ ]:
# from tqdm.auto import tqdm
# generated_subnarrative_texts = []

# for idx, row in tqdm(target_subnarratives_df.iterrows(), total=len(target_subnarratives_df)):
#     print(f"Generating subnarratives for label: {row['label']}")
#     texts = []
#     # Prepare input for the generator
#     input_dict = {
#         "subnarrative_name": row['label'],
#         "subnarrative_def": row['definition'],
#         "num_examples": NUM_EXAMPLES_PER_LABEL,
#         "subnarrative_example": row['examples'] if pd.notnull(row['examples']) else "",
#         "format_instructions": "Return only the generated text as a string."
#     }
    
#     result = subnarrative_generator.invoke(input_dict)
#     # Extract the generated text from the result
#     articles = result.articles  # Access the articles from the Pydantic object
#     texts = [article.generated_text for article in articles]  # Extract the generated text from each article
#     generated_subnarrative_texts.append(texts)
    
# # Add the generated texts to the DataFrame
# target_subnarratives_df['generated_texts'] = generated_subnarrative_texts

# # Prepare data for JSON export
# subnarrative_texts_json = []
# for i, row in target_subnarratives_df.iterrows():
#     subnarrative_texts_json.append({
#         "subnarrative": row["label"],
#         "generated_texts": row["generated_texts"]
#     })
# # Save to file
# with open("generated_subnarrative_texts.json", "w", encoding="utf-8") as f:
#     json.dump(subnarrative_texts_json, f, ensure_ascii=False, indent=2)